In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import problem_unittests as tests
import tarfile

cifar10_dataset_folder_path = 'cifar-10-batches-py'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('cifar-10-python.tar.gz'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            'cifar-10-python.tar.gz',
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open('cifar-10-python.tar.gz') as tar:
        tar.extractall()
        tar.close()


tests.test_folder_path(cifar10_dataset_folder_path)

In [ ]:
import helper
import numpy as np

cifar10_dataset_folder_path = 'cifar-10-batches-py'

# Explore the dataset
batch_id = 1
sample_id = 5
helper.display_stats(cifar10_dataset_folder_path, batch_id, sample_id)

In [ ]:
def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    # TODO: Implement Function
    x = x * 1.0 / 255
    return x
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_normalize(normalize)

In [ ]:
import pandas as pd, numpy as np
from sklearn.preprocessing import label_binarize

def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """
    # TODO: Implement Function
    x_one_hot_encode = label_binarize(np.array(x), classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    return x_one_hot_encode
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_one_hot_encode(one_hot_encode)

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)

In [1]:
import os, pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 32, 32

#valid
features_valid, labels_valid = pickle.load(open('preprocess_validation.p','rb'))
#test   
features_test, labels_test = pickle.load(open('preprocess_training.p','rb'))


#build the network
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [3]:
#train the model
for i in range(1,6):
    features_train, labels_train = pickle.load(open('preprocess_batch_'+str(i)+'.p','rb'))
    model.fit(features_train, labels_train, batch_size=128, nb_epoch=100, validation_data=(features_valid, labels_valid))
    #predict_train = model.predict(features_train, batch_size=128)
model.save('models/model.h5')
model.save_weights('models/model_weights.h5')

Train on 9000 samples, validate on 5000 samples
Epoch 1/100
9000/9000 [==============================] - 1s - loss: 2.3020 - acc: 0.1021 - val_loss: 2.2969 - val_acc: 0.1004
Epoch 2/100
9000/9000 [==============================] - 0s - loss: 2.2947 - acc: 0.1208 - val_loss: 2.2903 - val_acc: 0.1422
Epoch 3/100
9000/9000 [==============================] - 0s - loss: 2.2876 - acc: 0.1384 - val_loss: 2.2828 - val_acc: 0.1534
Epoch 4/100
9000/9000 [==============================] - 0s - loss: 2.2788 - acc: 0.1509 - val_loss: 2.2733 - val_acc: 0.1758
Epoch 5/100
9000/9000 [==============================] - 0s - loss: 2.2704 - acc: 0.1607 - val_loss: 2.2614 - val_acc: 0.1854
Epoch 6/100
9000/9000 [==============================] - 0s - loss: 2.2556 - acc: 0.1639 - val_loss: 2.2445 - val_acc: 0.1716
Epoch 7/100
9000/9000 [==============================] - 0s - loss: 2.2389 - acc: 0.1746 - val_loss: 2.2213 - val_acc: 0.1932
Epoch 8/100
9000/9000 [==============================] - 0s - loss: 2.

In [4]:
#test
predict_test = model.predict(features_test, batch_size=64, verbose=1)
test_acc= model.evaluate(features_test, labels_test, batch_size=64, verbose=1)
print(test_acc)

 9472/10000 [===========================>..] - ETA: 0s[1.2602118412017822, 0.65280000000000005]
